In [13]:
## Imports
import h5py
import os
import numpy as np
import numpy as np
from tensorflow.keras.models import load_model



In [9]:
## Load Data

# Percorso del file HDF5 da cui caricare i dati
load_path = "C:\\Users\\bsbar\\Desktop\\Tesi\\ThesisPlaques\\classificator_data.h5"

# Inizializza le liste per contenere i dati
loaded_class_images = []
loaded_class_labels = None
loaded_patients = None

# Apre il file HDF5 in modalità lettura
with h5py.File(load_path, 'r') as hf:
    # Itera su tutti i gruppi nel file HDF5
    for group_name in hf.keys():
        # Controlla se il nome del gruppo inizia con "patient_"
        if group_name.startswith("patient_"):
            group = hf[group_name]
            # Carica le immagini dal dataset 'images' all'interno del gruppo
            loaded_class_images.append(np.array(group['images']))
        elif group_name == "labels":
            # Carica le etichette dal dataset 'labels'
            loaded_class_labels = hf['labels'][:]
        elif group_name == "patients":
            # Carica i numeri dei pazienti dal dataset 'patients'
            loaded_patients = hf['patients'][:]

print(f"Dati caricati da {load_path}")
print(f"Numero di pazienti: {len(loaded_class_images)}")
print(f"Esempio di immagini per paziente: {loaded_class_images[0].shape}")


Dati caricati da C:\Users\bsbar\Desktop\Tesi\ThesisPlaques\classificator_data.h5
Numero di pazienti: 124
Esempio di immagini per paziente: (28, 64, 64)


In [46]:
# Carica il modello encoder
encoder = load_model("C:\\Users\\bsbar\\Desktop\\Tesi\\ThesisPlaques\\encoder_models\\encoder_model32.h5", compile=False)

# Funzione per ottenere le feature da una singola immagine
def get_features_from_image(image):

    image = image.astype('float32')
    image = image / 255.0
    # Ottieni le feature dall'encoder
    if len(image.shape) == 2:  # Se l'immagine è 64x64
        image = np.expand_dims(image, axis=-1)  # Aggiungi canale se necessario (per immagini in bianco e nero)
    image = np.expand_dims(image, axis=0)  # Aggiungi dimensione batch
    features = encoder.predict(image)
    features = np.squeeze(features)
    #print(features.shape)
    return np.array(features)

# Lista per salvare le feature delle immagini
patients = []

# Ottieni le feature per ogni immagine nella lista

for i in range(len(loaded_class_images)):
    features_list = []
    for img in loaded_class_images[i]:
        features = get_features_from_image(img)
        features_list.append(features)
    fetures_list = np.array(features_list)
    patients.append(features_list)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

TypeError: object of type 'numpy.float32' has no len()

In [ ]:
import pandas as pd
import numpy as np

def remove_correlated_features(encoded_features, threshold):
    # Trasforma l'array numpy in un DataFrame
    df = pd.DataFrame(encoded_features)

    # Calcola la matrice di correlazione
    corr_matrix = df.corr().abs()

    # Seleziona il triangolo superiore della matrice di correlazione
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Calcola la somma delle correlazioni per ogni feature
    correlation_sums = upper.sum()

    # Trova le feature con somma delle correlazioni superiore alla soglia
    to_drop = [column for column in correlation_sums.index if correlation_sums[column] > threshold]

    # Stampa le feature rimosse
    print(f"Feature rimosse: {to_drop}")

    # Elimina le feature
    df.drop(df.columns[to_drop], axis=1, inplace=True)

    return df

# Utilizza la funzione sulle tue feature codificate
new_features = remove_correlated_features(encoded_features, soglia)

In [ ]:
import pandas as pd
import numpy as np

def remove_correlated_features(patients_features, threshold):
    # Concatena tutti i vettori di feature in un unico DataFrame
    all_features = np.concatenate(patients_features)
    df = pd.DataFrame(all_features)

    # Calcola la matrice di correlazione
    corr_matrix = df.corr().abs()

    # Seleziona il triangolo superiore della matrice di correlazione
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Calcola la somma delle correlazioni per ogni feature
    correlation_sums = upper.sum()

    # Trova le feature con somma delle correlazioni superiore alla soglia
    to_drop = [column for column in correlation_sums.index if correlation_sums[column] > threshold]

    # Stampa le feature rimosse
    print(f"Feature rimosse: {to_drop}")

    # Elimina le feature
    df.drop(df.columns[to_drop], axis=1, inplace=True)

    return df.values

# Utilizza la funzione sulle tue feature codificate
new_features = remove_correlated_features(patients_features, soglia)
